In [ ]:
pip installe pandas

In [1]:
import pandas as pd

# Données : Object Class et Affectation
data = {
    "Object Class": [
        "ISI","CREDEMTEL","NG ZURICH","BANCS CASH","CUSTODY CORPORATE ACTION","PTF","CND","NULL","NETWORK",
        "SURVEILLANCE GTW","ITV","[EXP]","[API]","EAPI","FINALLEC","PSIC600D","PRISM","PC15H",
        "PFOBD","PFOBU","PFOBJ","APP-MDAU","PRO-PLI","BRSI","PRA0000D","PTG00D","PG2D","CTMA",
        "PMADJ","PL43D","PL41","PDM","PDM1014D","FCD","EXT","PADLD","EALJ","TELTRESM","PSTLJ",
        "CGXM","ACEJ","PACM","MADI","PRC1000D","PALID","CFPM","CONTROL-EM","PING","ZBXAGENT"
    ],
    "Affectation": [
        "Process","Réseau","Réseau","Réseau","Process","Process","Réseau","","Réseau",
        "Disponibilité","Data","Data","Data","Data","Data","Process","Process","Process",
        "Flux","Flux","Flux","Flux","Flux","Flux","Flux","Flux","Flux","Process",
        "Process","Process","Process","Process","Process","Process","Infra","Data","Data","Data","Disponibilité",
        "Data","Data","Process","Process","Process","Data","Process","Infra","Infra","Infra"
    ]
}

# Création du DataFrame
df = pd.DataFrame(data)

# Sauvegarde dans un fichier Excel
df.to_excel("object_class_affectations.xlsx", index=False)

print("✅ Fichier généré : object_class_affectations.xlsx")


ModuleNotFoundError: No module named 'pandas'

In [1]:
# ==============================
# 🔹 Installation propre
# ==============================
!pip install youtube-transcript-api --quiet

from youtube_transcript_api import YouTubeTranscriptApi
import re

# 🔹 URL de la vidéo
url = "https://www.youtube.com/watch?v=KCnorHy5FE4"
video_id = re.search(r"v=([a-zA-Z0-9_-]+)", url).group(1)

print(f"🎥 Video ID : {video_id}\n")

try:
    # Récupération de la transcription
    transcript = YouTubeTranscriptApi.get_transcript(video_id, languages=['fr', 'ar', 'en'])
    print("✅ Transcription récupérée avec succès !")
    
    # Assembler le texte
    full_text = " ".join([t['text'] for t in transcript])
    
    print(f"\n📊 Statistiques :")
    print(f"   - Segments : {len(transcript)}")
    print(f"   - Caractères : {len(full_text)}")
    print("\n" + "=" * 60)
    print("📄 Aperçu de la transcription :\n")
    print(full_text[:800] + "...\n")
    
    # Sauvegarder
    with open("transcription.txt", "w", encoding="utf-8") as f:
        f.write(full_text)
    
    print("=" * 60)
    print("✅ Transcription complète enregistrée dans 'transcription.txt'")
    
except Exception as e:
    print(f"❌ Erreur : {e}")
    print("\n🔍 Diagnostic :")
    
    # Vérifier les transcriptions disponibles
    try:
        transcript_list = YouTubeTranscriptApi.list_transcripts(video_id)
        print("📋 Transcriptions disponibles :")
        for transcript in transcript_list:
            print(f"   - {transcript.language} ({transcript.language_code}) - {'Auto' if transcript.is_generated else 'Manuelle'}")
    except Exception as e2:
        print(f"   Impossible de lister les transcriptions : {e2}")


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


🎥 Video ID : KCnorHy5FE4

❌ Erreur : type object 'YouTubeTranscriptApi' has no attribute 'get_transcript'

🔍 Diagnostic :
   Impossible de lister les transcriptions : type object 'YouTubeTranscriptApi' has no attribute 'list_transcripts'


In [2]:
# ==============================
# 🔹 Solution avec yt-dlp (LA PLUS FIABLE)
# ==============================

!pip install yt-dlp --quiet

import yt_dlp
import re

url = "https://www.youtube.com/watch?v=KCnorHy5FE4"

print("🔄 Téléchargement des sous-titres...\n")

ydl_opts = {
    'skip_download': True,
    'writesubtitles': True,
    'writeautomaticsub': True,
    'subtitleslangs': ['fr', 'ar', 'en'],
    'quiet': True,
    'no_warnings': True,
}

try:
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(url, download=False)
        
        # Vérifier les sous-titres disponibles
        subs = info.get('subtitles', {})
        auto_subs = info.get('automatic_captions', {})
        
        print("📋 Sous-titres disponibles :")
        all_subs = {**subs, **auto_subs}
        for lang in all_subs.keys():
            print(f"   - {lang}")
        
        print("\n🔄 Extraction de la transcription...\n")
        
        # Chercher dans l'ordre : fr, ar, en
        transcript_text = None
        selected_lang = None
        
        for lang in ['fr', 'ar', 'en']:
            if lang in subs:
                selected_lang = lang
                sub_data = subs[lang]
                break
            elif lang in auto_subs:
                selected_lang = lang
                sub_data = auto_subs[lang]
                break
        
        if selected_lang and sub_data:
            # Prendre le format JSON
            import requests
            
            for fmt in sub_data:
                if 'json3' in fmt.get('ext', ''):
                    sub_url = fmt['url']
                    response = requests.get(sub_url)
                    data = response.json()
                    
                    # Extraire le texte
                    texts = []
                    for event in data.get('events', []):
                        if 'segs' in event:
                            for seg in event['segs']:
                                if 'utf8' in seg:
                                    texts.append(seg['utf8'])
                    
                    transcript_text = " ".join(texts)
                    break
            
            if transcript_text:
                # Nettoyer le texte
                transcript_text = re.sub(r'\s+', ' ', transcript_text).strip()
                
                print(f"✅ Transcription extraite en {selected_lang.upper()}")
                print(f"📊 Longueur : {len(transcript_text)} caractères\n")
                print("=" * 60)
                print("📄 Aperçu :\n")
                print(transcript_text[:1000] + "...\n")
                print("=" * 60)
                
                # Sauvegarder
                with open("transcription.txt", "w", encoding="utf-8") as f:
                    f.write(transcript_text)
                
                print("\n✅ Transcription complète sauvegardée dans 'transcription.txt' !")
            else:
                print("❌ Impossible d'extraire le texte")
        else:
            print("❌ Aucun sous-titre disponible dans les langues demandées")
            
except Exception as e:
    print(f"❌ Erreur : {e}")
    import traceback
    traceback.print_exc()


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


🔄 Téléchargement des sous-titres...

📋 Sous-titres disponibles :
   - ab
   - aa
   - af
   - ak
   - sq
   - am
   - ar
   - hy
   - as
   - ay
   - az
   - bn
   - ba
   - eu
   - be
   - bho
   - bs
   - br
   - bg
   - my
   - ca
   - ceb
   - zh-Hans
   - zh-Hant
   - co
   - hr
   - cs
   - da
   - dv
   - nl
   - dz
   - en
   - eo
   - et
   - ee
   - fo
   - fj
   - fil
   - fi
   - fr-orig
   - fr
   - gaa
   - gl
   - lg
   - ka
   - de
   - el
   - gn
   - gu
   - ht
   - ha
   - haw
   - iw
   - hi
   - hmn
   - hu
   - is
   - ig
   - id
   - iu
   - ga
   - it
   - ja
   - jv
   - kl
   - kn
   - kk
   - kha
   - km
   - rw
   - ko
   - kri
   - ku
   - ky
   - lo
   - la
   - lv
   - ln
   - lt
   - lua
   - luo
   - lb
   - mk
   - mg
   - ms
   - ml
   - mt
   - gv
   - mi
   - mr
   - mn
   - mfe
   - ne
   - new
   - nso
   - no
   - ny
   - oc
   - or
   - om
   - os
   - pam
   - ps
   - fa
   - pl
   - pt
   - pt-PT
   - pa
   - qu
   - ro
   - rn
   - ru
   - sm

In [2]:
from youtube_transcript_api import YouTubeTranscriptApi

# Remplacez l'ID de la vidéo par l'ID de la vidéo YouTube que vous voulez transcrire
video_id = "KCnorHy5FE4"

try:
    # Récupère les sous-titres
    transcript = YouTubeTranscriptApi.fetch_transcript(video_id)

    # Crée le texte de la transcription
    text = " ".join([d['text'] for d in transcript])

    # Affiche le texte de la transcription
    print(text)

except Exception as e:
    print(f"Erreur: {e}")
    print("Assurez-vous que l'ID de la vidéo est correct et qu'elle a des sous-titres.")


Erreur: type object 'YouTubeTranscriptApi' has no attribute 'fetch_transcript'
Assurez-vous que l'ID de la vidéo est correct et qu'elle a des sous-titres.


In [4]:
# ==============================
# 🔹 Script corrigé pour extraire la transcription YouTube
# ==============================

!pip install youtube-transcript-api --upgrade

from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled, NoTranscriptFound
import re

# 🔹 URL de la vidéo
url = "https://www.youtube.com/watch?v=KCnorHy5FE4"

# Extraire l'identifiant de la vidéo
video_id = re.search(r"v=([a-zA-Z0-9_-]+)", url).group(1)

try:
    # Tenter d'obtenir la transcription en français
    transcript = YouTubeTranscriptApi().get_transcript(video_id, languages=['fr'])
except NoTranscriptFound:
    # Si pas de français, on récupère la première langue disponible
    available = YouTubeTranscriptApi().list_transcripts(video_id)
    transcript = available.find_transcript(available._manually_created_transcripts.keys() or available._generated_transcripts.keys()).fetch()
except TranscriptsDisabled:
    print("❌ Les sous-titres sont désactivés pour cette vidéo.")
    transcript = []
except Exception as e:
    print(f"⚠️ Erreur : {e}")
    transcript = []

# 🔹 Vérifier et assembler le texte
if transcript:
    full_text = " ".join([t['text'] for t in transcript])
    print("📄 Transcription (aperçu) :\n")
    print(full_text[:1000] + "...\n")

    # 🔹 Sauvegarder la transcription dans un fichier texte
    with open("transcription.txt", "w", encoding="utf-8") as f:
        f.write(full_text)
    print("✅ Transcription enregistrée dans 'transcription.txt'")
else:
    print("⚠️ Aucune transcription trouvée.")


⚠️ Erreur : 'YouTubeTranscriptApi' object has no attribute 'get_transcript'
⚠️ Aucune transcription trouvée.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip
